# This notebook will demonstrate the process of securely sharing data across regions using the Secure Sharing Area (SSA)

In [ ]:
import streamlit as st
st.image("XRegionShare.png")

In [ ]:
use role ACCOUNTADMIN; 
create or replace database CROSSREGIONSHARETEST; 
use schema CROSSREGIONSHARETEST.PUBLIC;

### Create our test data using SNOWFLAKE_SAMPLE_DATA

In [ ]:
create or replace table customer as 
select * from snowflake_sample_data.TPCDS_SF10TCL.customer;

In [ ]:
use schema CROSSREGIONSHARETEST.PUBLIC;

### Lets look at some of our data...

In [ ]:
select * from customer limit 10;

### Create a masking policy for character data... we will apply this later

In [ ]:
create or replace masking policy simple_mask_string as
  (val string) returns string ->
  case
    when current_role() in ('ACCOUNTADMIN') then val
      else '*** masked *****'
    end;

### Create a view on our customer table.  Imagine that the customer_id is sensitive so we'll encrypt it

In [ ]:
create or replace SECURE view customerV as 
select 
encrypt(c_customer_id,'abc') as c_customer_id_crypt, 
c_email_address,
c_first_name,
c_last_name, 
c_birth_year
from customer; 

select * from customerV limit 10;

### Test the masking policy. Use any role that is not ACCOUNTADMIN

In [ ]:
use role sysadmin; 
select * from customerV limit 10;

In [ ]:
use role accountadmin; 

### Apply the masking policy to specific columns in our view. The masking will only be in effect for roles other than ACCOUNTADMIN

In [ ]:
alter view customerV modify
  column c_email_address set masking policy simple_mask_string,
  column c_last_name set masking policy simple_mask_string,
  column c_first_name set masking policy simple_mask_string;

## Now lets share this view to the consumer account by creating and publishing a listing
1. Go to Data Products-> Provider Studio
2. Click +Create Listing (to create a new listing)
3. Select "Specified Consumers"
4. Name the Listing (ie. XREGIONSHARE_LISTING). Click Save
5. Click "Add data product" on the right
6. Click "+ Select"
7. Click the name of the database to share (CROSSREGIONSHARETEST)
8. Open "Secure View" and select CUSTOMERV and click "Done"
9. Change the name of the share identifier to something simpler like "XREGION_SHARE" and click "Save"
10. Click "Access Type" on the right and select "Free"
11. Add descriptive information in the other sections (Optional)
12. Select "Add a Consumer Account" under "Who can access"
    use the format ORGNAME.ACCOUNTNAME. If the Org/Account you specified exists in another region or cloud provider, you will be asked to specify a refresh interval for the your data product. When finished click "Save"
13. Specify Legal Terms (select "Terms of service provided offline")
14. You can click "Preview" to view the format of the listing as it will appear to the consumer. (Optional)
15. If everything looks good, click "Publish" and your listing will be LIVE!

To Adjust the refresh interval, go back into the listing in Provider Studio, scroll down to Consumer Accounts. Click "..." and Click "Update Refresh Frequency"

### Now, go to the consumer account
1. Click Data Products->Private Sharing
2. Look for listing under "Shared with You" and select "Get".  Should be XREGIONSHARE

At this point, you will see an indicator that the data is replicating to the local region.  This may take several minutes. 

3. When complete, provide a name for the shared database (XREGIONSHAREDB)
4. Execute the following in a worksheet or workspace on the consumer account

        use role accountadmin;
        
        use schema xregionshareDB.public;
        
        select * from customerV limit 10; 
        
Validate that the masking policy and encryption function are in effect!



### Note that the access for the consumer is revokable.  Revoke is immediate in the same region.  If different region, subject to the defined refresh interval specified in the listing

In [ ]:
revoke select on customerV from share XREGION_SHARE;

In [ ]:
grant select on customerV to share XREGION_SHARE;

### Cleanup...

In [ ]:
-- Unpublish and drop the listing from Provider Studio

drop database if exists CROSSREGIONSHARETEST; 